In [ ]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
pip install transformers

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import transformers

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
# 특정 파일 불러오기
import pandas as pd
#train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/test/train_preprocessed.csv")
train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청/train_preprocessed_dup.csv")
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청/test_preprocessed.csv")

tsv로 변환해보기

In [ ]:
test_preprocessed.head(3)

In [ ]:
#train_preprocessed[["document","label"]].to_csv("train_preprocessed.tsv",encoding="utf-8-sig",index=False,sep="\t")

max_seq_len = 64

In [ ]:
train_preprocessed["label"].value_counts()


In [ ]:
except_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 2].index
except_label
#1개인 label 제거

mask = train_preprocessed['label'].isin(except_label)
train_preprocessed = train_preprocessed[~mask].reset_index(drop=True)
train_preprocessed

In [ ]:
train_preprocessed["label"].value_counts()

label 데이터 인코딩하기

In [ ]:
train_preprocessed['encoded_cat'] = train_preprocessed['label'].astype('category').cat.codes

Label count 하기

KLUE 토크나이저

In [ ]:
MODEL_NAME = 'klue/roberta-base'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)


In [ ]:
print(tokenizer.tokenize("건물 내에서 고객의 요청에 따라 실내장식 공사 도배 등"))
print(tokenizer.encode("건물 내에서 고객의 요청에 따라 실내장식 공사 도배 등"))

In [ ]:
# 인코딩된 것 다시 디코딩 가능함
# 맨앞에 CLS 맨 뒤에 SEP
print(tokenizer.decode(tokenizer.encode("건물 내에서 고객의 요청에 따라 실내장식 공사 도배 등")))

max_seq_len와 인코딩 결과

In [ ]:
max_seq_len = 64
encoded_result = tokenizer.encode("건물 내에서 고객의 요청에 따라 실내장식 공사 도배 등",max_length = max_seq_len,padding='max_length') # max_seq_len으로 인코딩
print(encoded_result) #인코딩 결과 : 64짜리...

In [ ]:
# 세그먼트 인풋
print([0]*max_seq_len)

In [ ]:
# 마스크 인풋 : 토큰이 있는 부분은 1 나머지 토큰이 없는 부분은 0
valid_num = len(tokenizer.encode("건물 내에서 고객의 요청에 따라 실내장식 공사 도배 등"))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

데이터 to feature

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train_preprocessed['document'], train_preprocessed['encoded_cat'], test_size=0.2, random_state=777, stratify=train_preprocessed['encoded_cat'])
train_X, train_y = convert_examples_to_features(train_X, train_y, max_seq_len=max_seq_len, tokenizer=tokenizer)
val_X, val_y = convert_examples_to_features(val_X, val_y, max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
# 최대길이 64

input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

In [ ]:
import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel
model = TFRobertaModel.from_pretrained(MODEL_NAME, from_pt=True)

사전학습된 TFBertModel KLUE 불러오기

In [ ]:
max_seq_len = 64

각각의 레이어 설정

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [ ]:
class Klue_RobertaClassifier(tf.keras.Model):
    def __init__(self, num_class):
        super(Klue_RobertaClassifier, self).__init__()
        self.bert = TFRobertaModel.from_pretrained(MODEL_NAME, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_class,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range,seed=42), 
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
NUM_CLASSES = 223
model = Klue_RobertaClassifier(num_class=NUM_CLASSES)
optimizer = AdamWeightDecay(1e-5,weight_decay_rate=1e-4)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics = [metric])

In [ ]:
# Adding an ealrystop to prevent overfitting
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=2)

fit & predict
- max_seq_len이 64이기에 batch_size도 64로 설정
- https://peltarion.com/knowledge-center/documentation/cheat-sheets/bert---text-classification-/-cheat-sheet

In [ ]:
model_path = F"/content/MyDrive/MyDrive/통계청/"

checkpoint_path = os.path.join(model_path,'weight_klue_roberta_base_v1.h5')
cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
model.fit(train_X, train_y, epochs=5, batch_size=64, validation_data = (val_X,val_y), callbacks=[earlystop_callback, cp_callback])

In [ ]:
model_save_name = 'weight_klue_roberta_base_v1.h5'
path = F"/content/MyDrive/MyDrive/test/{model_save_name}" 
model.save_weights(path)

예측하기

In [ ]:
test_preprocessed["label"] = 0
test_X, test_y = convert_examples_to_features(test_preprocessed['document'], test_preprocessed['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
preds = model.predict(test_X)

In [ ]:
preds

In [ ]:
np.save("/content/MyDrive/MyDrive/통계청/weight_klue_roberta_base_v1.npy",preds)
# 값 저장
# np.load("/content/MyDrive/MyDrive/test/weight_klue_robertav0_large.npy")

label로 디코딩하기
- row를 딕셔너리로 : https://seong6496.tistory.com/109

In [ ]:
pred_label = tf.argmax(np.load("/content/MyDrive/MyDrive/통계청/weight_klue_roberta_base_v1.npy"),axis=1)

In [ ]:
pred_label

In [ ]:
# label 과 code 매핑 정보가 담긴 encoding dataframe
encoding = train_preprocessed[["label","encoded_cat"]].value_counts().reset_index()[["label","encoded_cat"]]
encoding.head(3)

In [ ]:
# 디코딩 작업
pred_label_lst = []

for code in tqdm(pred_label):
    label = encoding[encoding["encoded_cat"] == code].iloc[0,0]
    pred_label_lst.append(label)

### 제출형식에 맞게 만들어주기

In [ ]:
test_preprocessed["label"] = pred_label_lst

def make_predict(df):
# digit column 형성
    df["digit_1"] = df["label"].str[0]
    df["digit_2"] = df["label"].apply(lambda x : x[1:3] if len(x) == 6 else x[1])
    df["digit_3"] = df["label"].apply(lambda x : x[3:] if len(x) == 6 else x[-2:])

    # test_id column 형성
    test_id = []
    for i in range(1,100001):
        num = str(i).zfill(6)
        test_id.append("id_" + num)
    df["AI_id"] = test_id

    # 저장
    return df[["AI_id","digit_1","digit_2","digit_3","text_obj","text_mthd","text_deal"]]
    
    
make_predict(test_preprocessed).to_csv("/content/MyDrive/MyDrive/통계청/weight_klue_roberta_base_v1.csv",encoding="utf-8-sig",index=False)
                                                                                                
                                                                                                     

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)